In [2]:
import os
import sys
import pymongo
import pandas as pd
import plotly.express as px
import csv
import datetime
from datetime import datetime
import re

In [11]:
from pymongo import MongoClient

# Replace with your MongoDB connection string
database_url = os.getenv('db_url_for_creating')

# Connect to the MongoDB server
client = MongoClient(database_url)

# Access the specific database
db_name = "real_news"
db = client[db_name]

collection_name = "sampleCollection"
collection = db[collection_name]
dummy_data = [
    {"name": "Alice", "age": 25, "city": "New York"},
    {"name": "Bob", "age": 30, "city": "Los Angeles"},
]

# Insert dummy data into the collection
inserted_ids = collection.insert_many(dummy_data).inserted_ids

# List all collection names in the database
collection_names = db.list_collection_names()

# Print the collection names
print("Collections in the database:")
for name in collection_names:
    print(name)


Collections in the database:
sampleCollection


In [12]:
source_db_name = 'hack2'
target_db_name = db_name

# Get the source database
source_db = client[source_db_name]

# Create the target database
target_db = client[target_db_name]

# List of collections to keep non-empty
keep_non_empty = ['idstorages', 'selectedusers']

# List all collections in the source database
collections = source_db.list_collection_names()

# Copy collections to the target database
for collection_name in collections:
    source_collection = source_db[collection_name]
    target_collection = target_db[collection_name]

    # Copy documents if the collection should be kept non-empty
    if collection_name in keep_non_empty:
        # Copy documents from source collection to target collection
        for doc in source_collection.find():
            target_collection.insert_one(doc)
    else:
        # Create an empty collection in the target database
        target_collection.drop()  # Ensure the collection is empty
        target_db.create_collection(collection_name)

print(f"Database '{target_db_name}' created and collections copied. Collections '{', '.join(keep_non_empty)}' retained with data.")

Database 'real_news' created and collections copied. Collections 'idstorages, selectedusers' retained with data.


In [14]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import os
# Replace with your MongoDB connection string
database_url = os.getenv('db_url_for_creating')

# Connect to the MongoDB server
client = MongoClient(database_url)

# Access the specific database
db_name = 'real_news'
db = client[db_name]

three_weeks_ago = datetime.now() - timedelta(days=10) #datetime.now() - timedelta(weeks=3)

now = datetime.now()

# Calculate start and end of today
start_of_today = datetime(now.year, now.month, now.day)  # 00:00:00 today
end_of_today = start_of_today + timedelta(days=3)  # 00:00:00 tomorrow

# Calculate start of yesterday
start_of_yesterday = start_of_today - timedelta(days=20)
print(start_of_yesterday)
print(end_of_today)

# Iterate over all collections in the database
for collection_name in db.list_collection_names():
    collection = db[collection_name]
    
    # Assuming the documents have a field named 'created_at' or similar which stores the date
    # Replace 'created_at' with the appropriate field name in your collections
    result = collection.delete_many({"createdAt": {'$gte': start_of_yesterday,'$lt': end_of_today}})
    
    print(f"Deleted {result.deleted_count} documents from {collection_name}")

print("Data deletion complete.")

2024-10-25 00:00:00
2024-11-17 00:00:00
Deleted 0 documents from idstorages
Deleted 0 documents from postlikes
Deleted 0 documents from commentdislikes
Deleted 37 documents from posts
Deleted 0 documents from subscriptions
Deleted 20 documents from users
Deleted 0 documents from comments
Deleted 0 documents from viewposts
Deleted 0 documents from users5
Deleted 0 documents from sampleCollection
Deleted 0 documents from reposts
Deleted 0 documents from selectedusers
Deleted 0 documents from commentlikes
Deleted 0 documents from postdislikes
Deleted 0 documents from readposts
Data deletion complete.


In [13]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import os

# Replace with your MongoDB connection string
database_url = os.getenv('db_url_for_creating')

# Connect to the MongoDB server
client = MongoClient(database_url)

db = client["fake_news"]
users_collection = db["users"]     # Replace with your collection name

# Define the cutoff date
cutoff_date = datetime(2024, 11, 14, 9, 9, 59)

# Delete users created after the cutoff date
result = users_collection.delete_many({"createdAt": {"$gt": cutoff_date}})
print(f"Deleted {result.deleted_count} documents.")

Deleted 20 documents.


In [7]:

source_db = client["ms1"]  # Source database
target_db = client["ms3"]  # Target database

# Copy all documents from the "users" collection
users = list(source_db["users"].find())
if users:
    target_db["users"].insert_many(users)
    print(f"Copied {len(users)} users to ms4.")

# Copy only the first 20 documents from the "posts" collection
posts = list(source_db["posts"].find().limit(20))
if posts:
    target_db["posts"].insert_many(posts)
    print(f"Copied {len(posts)} posts to ms4.")

Copied 15 users to ms4.
Copied 20 posts to ms4.


In [12]:
from pymongo import MongoClient
import os

# Replace with your MongoDB connection string
database_url = os.getenv('db_url_for_creating')

# Connect to the MongoDB server
client = MongoClient(database_url)

db = client["fake_news"]
posts_collection = db["posts"]  # Replace with your collection name

# Find the first 150 documents based on the creation order
first_150_docs = posts_collection.find().sort("createdAt", 1).limit(150)

# Store the IDs of these documents
first_150_ids = [doc["_id"] for doc in first_150_docs]

# Delete all documents that are not in the first 150
result = posts_collection.delete_many({
    "_id": {"$nin": first_150_ids}
})

print(f"Deleted {result.deleted_count} documents.")



Deleted 3 documents.
